In [1]:
import os
os.chdir(os.pardir)

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns
import matplotlib.patches as mpatches
import pandas as pd

import vapor
from vapor.models import Cambium

2020-11-04 11:20:25,539 [MainThread  ] [INFO ]  Starting log for vapor...


In [2]:
os.getcwd()

'/srv/data/home/skoebric/vapor-open'

In [3]:
# --- Load Regional Best ---
results = pd.read_pickle(os.path.join('results','bnef_results.pkl'))
results['scenario'] = 'Mid Cost'
results['batt_size'] = 0
results.rename({'geometry':'point', 'region':'pca'}, axis='columns', inplace=True)


In [4]:
results['pca']

2        MA
3        MA
10      WNC
6       ENC
23      WSC
       ... 
1433    ENC
1438     NE
1439    WSC
1440    WSC
1443    ENC
Name: pca, Length: 975, dtype: object

In [5]:
region_map = pd.read_csv(os.path.join('data','geography','region_hierarchy.csv'))
region_map = region_map[['pca','rto','census_reg','inter']].drop_duplicates()

census_reg_dict = {
    'PA':'West',
    'MTN':'West',
    'ENC':'Midwest',
    'WNC':'Midwest',
    'WSC':'South',
    'MA':'Northeast',
    'NE':'Northeast',
    'ESC':'South',
    'SA':'South'
}
region_map['census_div'] = region_map['census_reg'].map(census_reg_dict)

df = results.merge(region_map, on='pca', how='left')


In [6]:
results['pca']

2        MA
3        MA
10      WNC
6       ENC
23      WSC
       ... 
1433    ENC
1438     NE
1439    WSC
1440    WSC
1443    ENC
Name: pca, Length: 975, dtype: object

In [7]:
years = [str(i) for i in range(2014,2051,1)]
cambium_cols = [i for i in df.columns if 'cambium' in i]
cambium_cols = [i for i in cambium_cols if i[-4:] in years]


# --- subset cambium columns ---
c_df = df[cambium_cols + list(region_map.columns) + ['project_id','tech','system_capacity']]

# --- melt ---
c_df = c_df.melt(id_vars= list(region_map.columns) +['project_id','tech','system_capacity'])
c_df['year'] = [int(i[-4:]) for i in c_df['variable']]
c_df['variable'] = [str(i[:-5]) for i in c_df['variable']]

c_df['value_mw'] = c_df['value'] / c_df['system_capacity'] * 1000


In [8]:
c_df

,pca,rto,census_reg,inter,census_div,project_id,tech,system_capacity,variable,value,year,value_mw
0,MA,NaN,NaN,NaN,NaN,607,wind,4500.0,cambium_total_value,175.989956,2020,39.108879
1,MA,NaN,NaN,NaN,NaN,607,wind,4500.0,cambium_total_value,175.989956,2020,39.108879
2,WNC,NaN,NaN,NaN,NaN,2691,wind,10000.0,cambium_total_value,829.288063,2020,82.928806
3,ENC,NaN,NaN,NaN,NaN,1502,wind,9000.0,cambium_total_value,779.711928,2020,86.634659
4,WSC,NaN,NaN,NaN,NaN,4775,wind,90800.0,cambium_total_value,8662.225784,2020,95.398962
...,...,...,...,...,...,...,...,...,...,...,...,...
170620,ENC,NaN,NaN,NaN,NaN,10000118,pv,23500.0,cambium_as_value,0.145177,2044,0.006178
170621,NE,NaN,NaN,NaN,NaN,10000122,pv,54600.0,cambium_as_value,0.239601,2044,0.004388
170622,WSC,NaN,NaN,NaN,NaN,10000123,pv,165000.0,cambium_as_value,50.271014,2044,0.304673
170623,WSC,NaN,NaN,NaN,NaN,10000124,pv,165000.0,cambium_as_value,50.271014,2044,0.304673


In [9]:

value_col = 'value'
grouped = c_df.groupby(['census_div','tech','year','variable'], as_index=False)[['system_capacity',value_col]].sum()
grouped['census_div'] = pd.Categorical(grouped['census_div'], categories=['Northeast','West','Midwest','South'], ordered=True)
grouped.sort_values('census_div', inplace=True)

grouped = grouped.loc[grouped['year'] == 2020]
grouped = grouped.loc[grouped['variable'].isin(['cambium_co2_rate_avg','cambium_grid_value'])]

pivot = grouped.pivot_table(
    index=['census_div','tech'],
    columns='variable',
    values='value'
)
pivot.reset_index(inplace=True)

sc = grouped[['census_div', 'tech', 'system_capacity']].drop_duplicates()

pivot = pivot.merge(sc, on=['tech','census_div'])

pivot['cambium_co2_rate_avg'] /= 1000000
pivot['cambium_grid_value']  /= 1e6
pivot['system_capacity'] /= 1000

pivot.to_csv(os.path.join('images','existing_impacts.csv'))
pivot

/home/skoebric/anaconda2/envs/vp/lib/python3.8/site-packages/pandas/core/generic.py:3878: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


KeyError: 'cambium_co2_rate_avg'

In [ ]:
grouped

In [ ]:
pivot

In [ ]:
dist = c_df.copy()
# dist['census_div'] = pd.Categorical(dist['census_div'], categories=['Northeast','West','Midwest','South'], ordered=True)
# dist.sort_values('census_div', inplace=True)


dist = dist.loc[dist['variable'].isin(variable_dict.keys())]

sns.set_style('whitegrid')
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10,4))

grid_value = dist.loc[dist['variable'] == 'cambium_grid_value']
co2 = dist.loc[dist['variable'] == 'cambium_co2_rate_avg']

sns.set_palette(sns.color_palette([vapor.tech_dict['pv'], vapor.tech_dict['wind']]))

sns.violinplot(
    data=grid_value, x='census_div', y='value_mw',
    hue='tech', split=True, inner='quart',
    ax=ax1, cut=0, scale='width'
  )

sns.violinplot(
    data=co2, x='census_div', y='value_mw',
    hue='tech', split=True, inner='quart', 
    ax=ax2, cut=0, scale='width'
  )

ax1.set_ylabel('2020 Grid Value per MW ($1,000)')
ax2.set_ylabel('2020 Average CO2 Offset per MW (lbs)')
ax1.set_xlabel('Census Division')
ax2.set_xlabel('Census Division')

ax1.legend([],[], frameon=False)
ax2.legend([],[], frameon=False)

label_dict = {'pv':'Solar PV', 'wind':'Wind'}
custom_patches = [mpatches.Patch(color=vapor.tech_dict[k], label=v) for k,v in label_dict.items()]
plt.legend(handles=custom_patches, bbox_to_anchor=(1.05, 0.6), loc=2, borderaxespad=0.)

plt.tight_layout()
plt.subplots_adjust(wspace=0.4)
sns.despine()

# g = sns.catplot(x='census_div', y='value_mw',
#                 hue='tech', col='variable',
#                data=dist, kind='violin', split=True, bw=.3,
#                height=5, aspect=1.2, sharey=False)


In [ ]:
dist

In [ ]:
df

In [ ]:
bars.pivot_table(index='census_div', columns='tech', values='value').plot.bar(stacked=True)

In [ ]:
# --- plot ---
g_df = c_df.groupby(['year','variable','tech'], as_index=False)['value'].sum()
g_df = g_df.loc[g_df['variable'] == 'cambium_total_value']
sns.lineplot(x='year', y='value', hue='tech', data=g_df)
